In [1]:
%%file total_energy.py

import pandas as pd

def import_data():
    csv_file = "../Resources/organised_Gen.csv"
    energy = pd.read_csv(csv_file)
    return energy

Overwriting total_energy.py


In [2]:
from total_energy import import_data

x = import_data()
x

,Unnamed: 0,YEAR,MONTH,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION (Megawatthours)
0,0,2001,1,AK,Total Electric Power Industry,Coal,46903.0
1,1,2001,1,AK,Total Electric Power Industry,Petroleum,71085.0
2,2,2001,1,AK,Total Electric Power Industry,Natural Gas,367521.0
3,3,2001,1,AK,Total Electric Power Industry,Hydroelectric Conventional,104549.0
4,4,2001,1,AK,Total Electric Power Industry,Wind,87.0
...,...,...,...,...,...,...,...
496769,10581,2022,5,WY,"Electric Generators, Electric Utilities",Coal,2071403.0
496770,10582,2022,5,WY,"Electric Generators, Electric Utilities",Hydroelectric Conventional,96790.0
496771,10583,2022,5,WY,"Electric Generators, Electric Utilities",Natural Gas,91570.0
496772,10584,2022,5,WY,"Electric Generators, Electric Utilities",Petroleum,1812.0


In [3]:
%%file tran_total_energy.py

from total_energy import import_data

import pandas as pd

def transform_data():
    
    tf_energy_data = import_data().drop(columns=['Unnamed: 0'])

    tf_energy_data = tf_energy_data.rename(columns={
        "YEAR": "year", 
        "MONTH": "month",
        "STATE": "state",
        "TYPE OF PRODUCER": "producer",
        "ENERGY SOURCE": "source",
        "GENERATION (Megawatthours)": "generated"})


    tf_energy_data['producer'] = tf_energy_data['producer'].apply(lambda x: x.replace(',','/'))
    tf_energy_data = tf_energy_data[tf_energy_data.state != 'US-TOTAL']
    tf_energy_data = tf_energy_data[tf_energy_data.producer != 'Total Electric Power Industry']
    tf_energy_data = tf_energy_data[tf_energy_data.source != 'Total']


    tf_energy_data = tf_energy_data.reset_index(drop = True)

    tf_energy_data = tf_energy_data.replace(to_replace = ['Wind','Solar Thermal and Photovoltaic', 
                                                 'Hydroelectric Conventional', 'Geothermal', 
                                                 'Wood and Wood Derived Fuels', 'Other Biomass', 
                                                 'Pumped Storage'],value = 'Renewable energy')
    tf_energy_data = tf_energy_data.groupby(['year','month','state','producer','source'],
                                            as_index=False).agg({'generated': 'sum'})
    
    return tf_energy_data

Overwriting tran_total_energy.py


In [4]:
from tran_total_energy import transform_data
x = transform_data()
x['source'].unique()

array(['Coal', 'Petroleum', 'Natural Gas', 'Renewable energy', 'Other',
       'Other Gases', 'Nuclear'], dtype=object)

In [5]:
from tran_total_energy import transform_data
import sys

import psycopg2

from psycopg2 import OperationalError, errorcodes, errors
import psycopg2.extras as extras
import pandas as pd
from io import StringIO
import numpy as np

import plotly.express as px
from timeit import default_timer as timer


from tran_total_energy import transform_data
start = timer()
x = transform_data()
end = timer()
k = end - start 
print("Query execution time: {}s".format(k))
x

Query execution time: 2.0902282139999997s


,year,month,state,producer,source,generated
0,2001,1,AK,Combined Heat and Power/ Commercial Power,Coal,8751.0
1,2001,1,AK,Combined Heat and Power/ Commercial Power,Petroleum,1073.0
2,2001,1,AK,Combined Heat and Power/ Electric Power,Coal,19742.0
3,2001,1,AK,Combined Heat and Power/ Electric Power,Petroleum,213.0
4,2001,1,AK,Combined Heat and Power/ Industrial Power,Natural Gas,62244.0
...,...,...,...,...,...,...
224657,2022,5,WY,Electric Generators/ Electric Utilities,Petroleum,1812.0
224658,2022,5,WY,Electric Generators/ Electric Utilities,Renewable energy,549703.0
224659,2022,5,WY,Electric Generators/ Independent Power Producers,Coal,55187.0
224660,2022,5,WY,Electric Generators/ Independent Power Producers,Natural Gas,0.0


In [6]:
%%file query_total_energy.py 

from tran_total_energy import transform_data
import sys

import psycopg2

from psycopg2 import OperationalError, errorcodes, errors
import psycopg2.extras as extras
import pandas as pd
from io import StringIO
import numpy as np

import plotly.express as px
from timeit import default_timer as timer

edf = transform_data()

params_dic = {
    "host"      : "localhost",
    "user"      : "postgres",
    "password"  : "postgres",
    "port"      : "5432"
}

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn
conn = connect(params_dic)

#install psycopg2-binary for MacOs if you don't have it
    #!pip install psycopg2-binary

# Define a function that handles and parses psycopg2 exceptions
def show_psycopg2_exception(err):
    # get details about the exception
    err_type, err_obj, traceback = sys.exc_info()    
    # get the line number when exception occured
    line_n = traceback.tb_lineno    
    # print the connect() error
    print ("\npsycopg2 ERROR:", err, "on line number:", line_n)
    print ("psycopg2 traceback:", traceback, "-- type:", err_type) 
    # psycopg2 extensions.Diagnostics object attribute
    print ("\nextensions.Diagnostics:", err.diag)    
    # print the pgcode and pgerror exceptions
    print ("pgerror:", err.pgerror)
    print ("pgcode:", err.pgcode, "\n")

def create_table(cursor):
    try:
        # 
        cursor.execute("DROP TABLE IF EXISTS energy_type;")
        sql = '''CREATE TABLE energy_type(
        year INT NOT NULL,
        month INT NOT NULL,
        state VARCHAR NOT NULL, 
        producer VARCHAR NOT NULL,
        source VARCHAR NOT NULL, 
        generated FLOAT NOT NULL
        )'''
        # Creating a table
        cursor.execute(sql);
        print("energy table is created successfully...............")  
    except OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None

# Define function using copy_from() with StringIO to insert the dataframe
def copy_from_dataFile_StringIO(conn, datafrm, table):
    
    #save dataframe to an in memory buffer
    buffer = StringIO()
    datafrm.to_csv(buffer, header=False, index = False)
    buffer.seek(0)
    
    cursor = conn.cursor()
    try:
        cursor.copy_from(buffer, table, sep=",")
        print("Data inserted using copy_from_datafile_StringIO() successfully....")
    except (Exception, psycopg2.DatabaseError) as err:
        # pass exception to function
        show_psycopg2_exception(err)
        cursor.close()

conn = connect(params_dic)

# We set autocommit=True so every command we execute will produce results immediately.
conn.autocommit = True
cursor = conn.cursor()
create_table(cursor)

copy_from_dataFile_StringIO(conn, edf, 'energy_type')


def query_data():
    start = timer()

    conn.autocommit = True
    cursor = conn.cursor()
  
    sql = '''SELECT state, source, SUM(generated)
            FROM energy_type 
            WHERE YEAR != 2022
            Group By state, source
            Order By state
                ;''' 
  
    cursor.execute(sql)
    results = cursor.fetchall()
    df = pd.DataFrame (results, columns = ['State', 'Energy Source','Total Generated'])
    conn.commit()
    end = timer()
    k = end - start 
    #print("Query execution time: {}s".format(k))
    #fig.show()
    return df

Overwriting query_total_energy.py


In [7]:
from tran_total_energy import transform_data
from query_total_energy import query_data

start = timer()
x = transform_data()
y = query_data()
end = timer()
k = end - start 
print("Query execution time: {}s".format(k))

Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful
energy table is created successfully...............
Data inserted using copy_from_datafile_StringIO() successfully....
Query execution time: 1.8420055820000023s


In [8]:
query_data()

,State,Energy Source,Total Generated
0,AK,Coal,1.328460e+07
1,AK,Natural Gas,7.167660e+07
2,AK,Other,5.767160e+03
3,AK,Petroleum,1.873001e+07
4,AK,Other Gases,5.396770e+03
...,...,...,...
318,WY,Other,1.379856e+06
319,WY,Renewable energy,8.200103e+07
320,WY,Natural Gas,1.364332e+07
321,WY,Coal,8.662618e+08


In [9]:
%%file test_total_energy.py

from total_energy import import_data
from tran_total_energy import transform_data
from query_total_energy import query_data

def test_data_columns_count():
    df = import_data()
    assert len(df.columns) == 7

def test_data_row_count():
    df = import_data()
    assert len(df.index) == 496774
    
def test_transform_data_columns_count():
    df = transform_data()
    assert len(df.columns) == 6

def test_transform_data_row_count():
    df = transform_data()
    assert len(df.index) == 224662
    
def test_query_data_column_count():
    df = query_data()
    assert len(df.columns) == 3

def test_query_data_row_count():
    df = query_data()
    assert len(df.index) == 323

Overwriting test_total_energy.py


In [10]:
# Run the test_import_data.py file with pytest. 
!python -m pytest test_total_energy.py

============================= test session starts ==============================
platform darwin -- Python 3.9.7, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: /Users/slimykid/Group4_NFLX_MIDTERM/Lisa
plugins: anyio-2.2.0
collected 6 items                                                              

test_total_energy.py ......                                              [100%]

============================== 6 passed in 12.68s ==============================


In [18]:
k =query_data()
fig = px.bar(k, x="State", y="Total Generated", 
             color='Energy Source', title ="Energy Generated by Each State")
fig.update_xaxes(dtick=1)
fig.show()